Anthony Bassil GIS Dev 08/24/2023

## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
import time
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
#Annonymous AGOL USER
gis = GIS()

In [2]:
#Get Content By Item ID
item = gis.content.get("a6134ae01aad44c499d12feec782b386")
flayer = item.layers[0]



In [3]:
#Verify that is the right one
item

<Item title:"USA Weather Watches and Warnings" type:Feature Layer Collection owner:esri_livefeeds2>

In [4]:
#Inspect URL REST Endpoints
for index, lyr in enumerate (item.layers):
    print(index,lyr.properties.name, lyr.url, lyr.properties.id)
    

0 Public Forecast Zones https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NWS_Watches_Warnings_v1/FeatureServer/1 1
1 Fire Forecast Zones https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NWS_Watches_Warnings_v1/FeatureServer/2 2
2 US Counties https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NWS_Watches_Warnings_v1/FeatureServer/3 3
3 US States and Territories https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NWS_Watches_Warnings_v1/FeatureServer/4 4
4 Coastal and Offshore Marine Zones https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NWS_Watches_Warnings_v1/FeatureServer/5 5
5 Events Ordered by Size and Severity https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NWS_Watches_Warnings_v1/FeatureServer/6 6
6 Extreme Events https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NWS_Watches_Warnings_v1/FeatureServer/8 8
7 Severe Events https://services9.arcgis.com/RHVPKKiFTONKt

In [5]:
#Create Spatial Dataframes

StatesSDF = pd.DataFrame.spatial.from_layer(item.layers[3])
CountiesSDF = pd.DataFrame.spatial.from_layer(item.layers[2])
EventsSDF = pd.DataFrame.spatial.from_layer(item.layers[5])

In [6]:
#First function takes a string seperated by whitespace and makes it a list
#Second function splits a string at Nth character and replaces it with desired Character
def convertAreaIds2list(xp):
    xp = xp.split()
    return xp
def replace_by_index(string, idx, character):
    return string[:idx] + character + string[idx+1:]


#Make a new column called Results and turn the AreaID into a list at each row
EventsSDF['Results0'] =  EventsSDF.apply(lambda x: convertAreaIds2list (str(x['AreaIds'])), axis=1)


In [7]:
#Create a table to join to by exploding the columns we changed from list to individual rows for each
NextLevel = EventsSDF.explode('Results0')
#Need to modify AreaIDs so that they can match the counties Spatial Dataframe
NextLevel['Swapped'] =  NextLevel.apply(lambda x: replace_by_index (str(x['Results0']),2,'C'), axis=1)

In [8]:
#Function to get first characters of string to make a State Abreviation Column
def parseStateAbbrev(lefttwo):
    return lefttwo[0:2]


In [9]:
#rename some columns to make sense to me
StatesSDF = StatesSDF.rename(columns={'UGC': 'StAbbv', 'Name': 'State'})
CountiesSDF = CountiesSDF.rename(columns={'UGC': 'CountyCode', 'Name': 'County'})
#Parse the CountyCode to get the state Abbeviation
CountiesSDF['StateAbbv'] =  CountiesSDF.apply(lambda x: parseStateAbbrev (str(x['CountyCode'])), axis=1)
#Join everthing into one big table
CountyStates = pd.merge(CountiesSDF,StatesSDF, left_on='StateAbbv', right_on='StAbbv', how='inner')
CountyStatesEvents= pd.merge(NextLevel, CountyStates, left_on='Swapped', right_on='CountyCode', how='inner')


In [10]:
#Make a list of active states and turn it into a dataframe
a = CountyStatesEvents['State'].unique()
Statelist = pd.DataFrame(a, columns=['Active_States'])
Statelist

,Active_States
0,South Dakota
1,Missouri
2,Oklahoma
3,Louisiana
4,Indiana
5,Illinois
6,Kansas
7,Kentucky
8,Mississippi
9,Arkansas


In [11]:
#This function returns two dataframes in the form of list, the first item is all the events for the state, the second item is all the counties
def countyQuery (Eventsdf, StateName):    
    WeatherAlert = Eventsdf[(Eventsdf['State'].str.contains('{}'.format( StateName)))]
    CountyList = (WeatherAlert['County']+', '+WeatherAlert['StateAbbv']).unique()
    CountyListDf = pd.DataFrame(CountyList, columns=['Active_Counties'])

    if WeatherAlert.shape[0] > 0:
        return [WeatherAlert,CountyListDf]
    elif WeatherAlert.shape[0] == 0:
        return ['Please Check State Spelling or there are no Adverse Weather Events','Please Check State Spelling or there are no Adverse Weather Events']


In [12]:
IndividualCounty = countyQuery(CountyStatesEvents,'West Virginia')
IndividualCounty[0]

,OBJECTID,Event,Severity,Summary,Link,Urgency,Certainty,Category,Updated,Start,...,Shape__Length_x,SHAPE_x,StateAbbv,OBJECTID_y,State,StAbbv,SGC_y,Shape__Area_y,Shape__Length_y,SHAPE_y
711,881613,Heat Advisory,Minor,...HEAT ADVISORY IN EFFECT FROM 2 PM TO 8 PM E...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 23:49:00,2023-08-24 23:49:00,...,279565.486668,"{'rings': [[[-9100635.05266184, 4608016.693879...",WV,50,West Virginia,WV,USS,1.030610e+11,2.474510e+06,"{'rings': [[[-8963467.10675433, 4959079.688361..."
712,881613,Heat Advisory,Minor,...HEAT ADVISORY IN EFFECT FROM 2 PM TO 8 PM E...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 23:49:00,2023-08-24 23:49:00,...,205746.714531,"{'rings': [[[-8972250.57068896, 4706186.306609...",WV,50,West Virginia,WV,USS,1.030610e+11,2.474510e+06,"{'rings': [[[-8963467.10675433, 4959079.688361..."
713,881613,Heat Advisory,Minor,...HEAT ADVISORY IN EFFECT FROM 2 PM TO 8 PM E...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 23:49:00,2023-08-24 23:49:00,...,183693.831015,"{'rings': [[[-9031116.42473244, 4727978.159513...",WV,50,West Virginia,WV,USS,1.030610e+11,2.474510e+06,"{'rings': [[[-8963467.10675433, 4959079.688361..."
714,881614,Heat Advisory,Minor,...HEAT ADVISORY IN EFFECT FROM 2 PM TO 8 PM E...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 23:49:00,2023-08-25 18:00:00,...,166622.816899,"{'rings': [[[-9013705.76827753, 4664181.409245...",WV,50,West Virginia,WV,USS,1.030610e+11,2.474510e+06,"{'rings': [[[-8963467.10675433, 4959079.688361..."
715,881614,Heat Advisory,Minor,...HEAT ADVISORY IN EFFECT FROM 2 PM TO 8 PM E...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 23:49:00,2023-08-25 18:00:00,...,350779.077811,"{'rings': [[[-8945979.16407127, 4595572.730594...",WV,50,West Virginia,WV,USS,1.030610e+11,2.474510e+06,"{'rings': [[[-8963467.10675433, 4959079.688361..."


In [13]:
IndividualCounty[1]

,Active_Counties
0,"Boone, WV"
1,"Braxton, WV"
2,"Calhoun, WV"
3,"Clay, WV"
4,"Greenbrier, WV"


In [14]:
IndividualCounty = countyQuery(CountyStatesEvents,'Texas')
IndividualCounty[0]

,OBJECTID,Event,Severity,Summary,Link,Urgency,Certainty,Category,Updated,Start,...,Shape__Length_x,SHAPE_x,StateAbbv,OBJECTID_y,State,StAbbv,SGC_y,Shape__Area_y,Shape__Length_y,SHAPE_y
63,879561,Excessive Heat Warning,Moderate,...EXCESSIVE HEAT WARNING REMAINS IN EFFECT UN...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 19:55:00,2023-08-24 19:55:00,...,3.539115e+05,"{'rings': [[[-10795796.864244, 4021232.8914277...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."
64,879561,Excessive Heat Warning,Moderate,...EXCESSIVE HEAT WARNING REMAINS IN EFFECT UN...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 19:55:00,2023-08-24 19:55:00,...,4.990935e+05,"{'rings': [[[-11626474.5704921, 3622780.776608...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."
65,879561,Excessive Heat Warning,Moderate,...EXCESSIVE HEAT WARNING REMAINS IN EFFECT UN...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 19:55:00,2023-08-24 19:55:00,...,3.185076e+05,"{'rings': [[[-11372732.5128543, 4308221.385425...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."
66,879561,Excessive Heat Warning,Moderate,...EXCESSIVE HEAT WARNING REMAINS IN EFFECT UN...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 19:55:00,2023-08-24 19:55:00,...,2.331421e+05,"{'rings': [[[-11189501.0658225, 3948200.216386...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."
67,879561,Excessive Heat Warning,Moderate,...EXCESSIVE HEAT WARNING REMAINS IN EFFECT UN...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-24 19:55:00,2023-08-24 19:55:00,...,3.914667e+05,"{'rings': [[[-11104586.3218028, 3540960.175556...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,881749,Heat Advisory,Minor,...HEAT ADVISORY REMAINS IN EFFECT FROM 10 AM ...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-25 01:25:00,2023-08-25 01:25:00,...,6.719861e+04,"{'rings': [[[-10599626.6388062, 3468209.224202...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."
687,881358,Red Flag Warning,Severe,...FIRE DANGER CONTINUES FRIDAY... Critically ...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-25 01:25:00,2023-08-25 01:25:00,...,1.883712e+05,"{'rings': [[[-10955384.6397548, 3495545.775445...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."
688,881387,Excessive Heat Warning,Moderate,...EXCESSIVE HEAT WARNING REMAINS IN EFFECT FR...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-25 01:25:00,2023-08-25 01:25:00,...,1.883712e+05,"{'rings': [[[-10955384.6397548, 3495545.775445...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."
689,881358,Red Flag Warning,Severe,...FIRE DANGER CONTINUES FRIDAY... Critically ...,https://alerts.weather.gov/cap/wwacapget.php?x...,Expected,Likely,Met,2023-08-25 01:25:00,2023-08-25 01:25:00,...,1.563027e+06,"{'rings': [[[-10833146.8606129, 3073933.277881...",TX,43,Texas,TX,USS,9.477483e+11,1.366283e+07,"{'rings': [[[-10819890.7130103, 2999379.559207..."


In [15]:
IndividualCounty[1]

,Active_Counties
0,"Cooke, TX"
1,"Culberson, TX"
2,"Dallam, TX"
3,"Dickens, TX"
4,"Edwards, TX"
...,...
67,"Howard, TX"
68,"Hemphill, TX"
69,"Harris, TX"
70,"Kendall, TX"


In [16]:
IndividualCounty = countyQuery(CountyStatesEvents,'Vermont')
IndividualCounty[0]

'Please Check State Spelling or there are no Adverse Weather Events'

In [17]:
IndividualCounty[1]

'Please Check State Spelling or there are no Adverse Weather Events'